In [11]:
"""
This is an example using CLAP to perform zeroshot
    classification on ESC50 (https://github.com/karolpiczak/ESC-50).
"""

from msclap import CLAP
from esc50_dataset import ESC50
import torch.nn.functional as F
import numpy as np
from tqdm import tqdm
from sklearn.metrics import accuracy_score

%load_ext autoreload
%autoreload 2

In [13]:
# Load dataset
root_path = "root_path" # Folder with ESC-50-master/
dataset = ESC50(root=root_path, download=True) #If download=False code assumes base_folder='ESC-50-master' in esc50_dataset.py
prompt = 'this is the sound of '
y = [prompt + x for x in dataset.classes]

Using downloaded and verified file: root_path/ESC-50-master.zip
Loading audio files


2000it [00:00, 9458.87it/s]


In [17]:
print(y)


['this is the sound of airplane', 'this is the sound of breathing', 'this is the sound of brushing teeth', 'this is the sound of can opening', 'this is the sound of car horn', 'this is the sound of cat', 'this is the sound of chainsaw', 'this is the sound of chirping birds', 'this is the sound of church bells', 'this is the sound of clapping', 'this is the sound of clock alarm', 'this is the sound of clock tick', 'this is the sound of coughing', 'this is the sound of cow', 'this is the sound of crackling fire', 'this is the sound of crickets', 'this is the sound of crow', 'this is the sound of crying baby', 'this is the sound of dog', 'this is the sound of door wood creaks', 'this is the sound of door wood knock', 'this is the sound of drinking sipping', 'this is the sound of engine', 'this is the sound of fireworks', 'this is the sound of footsteps', 'this is the sound of frog', 'this is the sound of glass breaking', 'this is the sound of hand saw', 'this is the sound of helicopter', 

In [15]:
# Load and initialize CLAP
clap_model = CLAP(version = '2023', use_cuda=False)

# Computing text embeddings
text_embeddings = clap_model.get_text_embeddings(y)

In [27]:
# Computing audio embeddings
y_preds, y_labels = [], []

for i in tqdm(range(len(dataset))):
    x, _, one_hot_target = dataset.__getitem__(i)
    print(one_hot_target)
    audio_embeddings = clap_model.get_audio_embeddings([x], resample=True)
    similarity = clap_model.compute_similarity(audio_embeddings, text_embeddings)
    y_pred = F.softmax(similarity.detach().cpu(), dim=1).numpy()
    print(y_pred)
    y_preds.append(y_pred)
    y_labels.append(one_hot_target.detach().cpu().numpy())
    break

  0%|          | 0/2000 [00:00<?, ?it/s]

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
[[1.66954374e-06 5.19530404e-05 1.47015162e-04 8.23233335e-04
  4.10466528e-05 1.08483369e-02 1.14987663e-03 5.60870627e-03
  6.17996993e-05 1.09129734e-02 8.31605867e-05 4.55333175e-05
  2.29639802e-04 1.85293949e-03 4.01197349e-06 6.12806587e-04
  2.77046040e-02 3.27465386e-05 8.92622113e-01 1.36927056e-05
  1.79056471e-04 2.75679067e-05 5.42990301e-06 9.94071524e-05
  1.75833411e-05 6.38487225e-04 2.35644355e-03 3.51600611e-04
  2.66799702e-06 9.98461153e-03 6.97230687e-04 2.00975137e-05
  1.59933566e-04 6.82282989e-05 1.21546665e-03 1.19438555e-05
  1.15009934e-05 1.76603869e-02 3.61806110e-06 2.86239083e-03
  4.68865401e-05 1.05483457e-02 1.34564525e-05 4.96565990e-05
  7.05740013e-06 3.76787239e-06 7.48499151e-05 4.11082146e-06
  1.63742279e-05 1.40533575

In [31]:
print(y_preds)
print(y_labels)


[array([[1.66954374e-06, 5.19530404e-05, 1.47015162e-04, 8.23233335e-04,
        4.10466528e-05, 1.08483369e-02, 1.14987663e-03, 5.60870627e-03,
        6.17996993e-05, 1.09129734e-02, 8.31605867e-05, 4.55333175e-05,
        2.29639802e-04, 1.85293949e-03, 4.01197349e-06, 6.12806587e-04,
        2.77046040e-02, 3.27465386e-05, 8.92622113e-01, 1.36927056e-05,
        1.79056471e-04, 2.75679067e-05, 5.42990301e-06, 9.94071524e-05,
        1.75833411e-05, 6.38487225e-04, 2.35644355e-03, 3.51600611e-04,
        2.66799702e-06, 9.98461153e-03, 6.97230687e-04, 2.00975137e-05,
        1.59933566e-04, 6.82282989e-05, 1.21546665e-03, 1.19438555e-05,
        1.15009934e-05, 1.76603869e-02, 3.61806110e-06, 2.86239083e-03,
        4.68865401e-05, 1.05483457e-02, 1.34564525e-05, 4.96565990e-05,
        7.05740013e-06, 3.76787239e-06, 7.48499151e-05, 4.11082146e-06,
        1.63742279e-05, 1.40533575e-05]], dtype=float32)]
[array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
    

In [ ]:
y_labels, y_preds = np.concatenate(y_labels, axis=0), np.concatenate(y_preds, axis=0)
acc = accuracy_score(np.argmax(y_labels, axis=1), np.argmax(y_preds, axis=1))
print('ESC50 Accuracy {}'.format(acc))

"""
The output:

ESC50 Accuracy: 93.9%

"""